In [2]:
%pip install flask flask-jwt-extended redis sqlalchemy


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [8]:
# Step 1: Imports and Configuration
import redis
from flask import Flask, request, jsonify
from flask_jwt_extended import JWTManager, create_access_token, jwt_required, get_jwt_identity
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import declarative_base
from sqlalchemy.orm import sessionmaker
from datetime import timedelta

app = Flask(__name__)
app.config['JWT_SECRET_KEY'] = 'secret-key'
app.config['JWT_ACCESS_TOKEN_EXPIRES'] = timedelta(hours=1)

jwt = JWTManager(app)
r = redis.Redis(host='localhost', port=6379, db=0, decode_responses=True)

# SQLite Setup
engine = create_engine('sqlite:///users.db')
Base = declarative_base()
SessionLocal = sessionmaker(bind=engine)
db = SessionLocal()

# Step 2: User Table Model
class User(Base):
    __tablename__ = "users"
    id = Column(Integer, primary_key=True)
    username = Column(String, unique=True, nullable=False)
    password = Column(String, nullable=False)

Base.metadata.create_all(engine)

# Step 3: Register Endpoint
@app.route("/register", methods=["POST"])
def register():
    data = request.get_json()
    if db.query(User).filter_by(username=data["username"]).first():
        return jsonify({"msg": "User already exists"}), 400
    user = User(username=data["username"], password=data["password"])
    db.add(user)
    db.commit()
    return jsonify({"msg": "User registered successfully"})

# Step 4: Login Endpoint
@app.route("/login", methods=["POST"])
def login():
    data = request.get_json()
    user = db.query(User).filter_by(username=data["username"], password=data["password"]).first()
    if not user:
        return jsonify({"msg": "Invalid credentials"}), 401
    token = create_access_token(identity=user.username)
    return jsonify(access_token=token)

# Step 5: Submit Score
@app.route("/submit_score", methods=["POST"])
@jwt_required()
def submit_score():
    current_user = get_jwt_identity()
    data = request.get_json()
    game = data["game"]
    score = data["score"]
    r.zadd(f"leaderboard:{game}", {current_user: score})
    return jsonify({"msg": f"Score {score} submitted for {current_user} in {game}."})

# Step 6: Get Global Leaderboard
@app.route("/leaderboard/<game>", methods=["GET"])
def get_leaderboard(game):
    leaderboard = r.zrevrange(f"leaderboard:{game}", 0, 9, withscores=True)
    return jsonify({"leaderboard": leaderboard})

# Step 7: Get User Rank
@app.route("/rank/<game>", methods=["GET"])
@jwt_required()
def get_rank(game):
    current_user = get_jwt_identity()
    rank = r.zrevrank(f"leaderboard:{game}", current_user)
    score = r.zscore(f"leaderboard:{game}", current_user)
    return jsonify({"user": current_user, "rank": rank + 1 if rank is not None else None, "score": score})

# Step 8: Run Server (in Jupyter)
from threading import Thread

def run_server():
    app.run(port=5000)

Thread(target=run_server).start()


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
